<a href="https://colab.research.google.com/github/amalabderrahmani/Colab_Project/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In this challenge, a machine learning model that will classify SMS messages as either "ham" or "spam" is created. A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.



For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.10.0-dev20220407


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-13 00:04:49--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-04-13 00:04:49 (11.4 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2022-04-13 00:04:49--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

# Datasets preprocessing

In [ ]:
train_dataset= pd.read_csv(train_file_path, sep='\t', names=["label", "message"])
test_dataset= pd.read_csv(test_file_path, sep= '\t',names=["label", "message"])

In [ ]:
print(train_dataset.shape[0])
print(test_dataset.shape[0])

4179
1392


In [ ]:
dataset=pd.concat([test_dataset, train_dataset])

In [ ]:
dataset.describe()

,label,message
count,5571,5571
unique,2,5156
top,ham,"sorry, i'll call later"
freq,4824,30


In [ ]:
dataset.groupby('label').describe().T

label                              ham  \
message count                     4824   
        unique                    4514   
        top     sorry, i'll call later   
        freq                        30   

label                                                        spam  
message count                                                 747  
        unique                                                642  
        top     please call our customer service representativ...  
        freq                                                    4

since the training data is obviously imbalanced, we downsample the ham messages to get a balanced data

In [ ]:
ham_messages= dataset[dataset.label == 'ham']
spam_messages= dataset[dataset.label == 'spam']
ham_msg_df = ham_messages.sample(n = len(spam_messages), random_state = 44)
spam_msg_df = spam_messages
print(ham_msg_df.shape, spam_msg_df.shape)

(747, 2) (747, 2)


In [ ]:
df = ham_msg_df.append(spam_msg_df).reset_index(drop=True)

In [ ]:
df

,label,message
0,ham,these won't do. have to move on to morphine
1,ham,love you aathi..love u lot..
2,ham,that's cause your old. i live to be high.
3,ham,no i'm in the same boat. still here at my moms...
4,ham,:( but your not here....
...,...,...
1489,spam,free msg: single? find a partner in your area!...
1490,spam,free2day sexy st george's day pic of jordan!tx...
1491,spam,you are a winner you have been specially selec...
1492,spam,free entry into our £250 weekly competition ju...


In [ ]:
# Get length column for each text
df['text_length'] = df['message'].apply(len)
#Calculate average length by label types
labels = df.groupby('label').mean()
labels

,text_length
label,
ham,70.716198
spam,138.601071


In [ ]:
text_length=df.pop('text_length')

In [ ]:
text_length

0        43
1        28
2        41
3        83
4        24
       ... 
1489    151
1490    154
1491    157
1492    106
1493    152
Name: text_length, Length: 1494, dtype: int64

In [ ]:
df

,label,message
0,ham,these won't do. have to move on to morphine
1,ham,love you aathi..love u lot..
2,ham,that's cause your old. i live to be high.
3,ham,no i'm in the same boat. still here at my moms...
4,ham,:( but your not here....
...,...,...
1489,spam,free msg: single? find a partner in your area!...
1490,spam,free2day sexy st george's day pic of jordan!tx...
1491,spam,you are a winner you have been specially selec...
1492,spam,free entry into our £250 weekly competition ju...


In [ ]:
df['label']= df['label'].map({'ham': 0, 'spam': 1})
df

,label,message
0,0,these won't do. have to move on to morphine
1,0,love you aathi..love u lot..
2,0,that's cause your old. i live to be high.
3,0,no i'm in the same boat. still here at my moms...
4,0,:( but your not here....
...,...,...
1489,1,free msg: single? find a partner in your area!...
1490,1,free2day sexy st george's day pic of jordan!tx...
1491,1,you are a winner you have been specially selec...
1492,1,free entry into our £250 weekly competition ju...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_text, test_text, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=434)

In [ ]:
train_text.shape

(1195,)

In [ ]:
test_text.shape

(299,)

In [ ]:
y_train

794     1
1074    1
1108    1
1085    1
1315    1
       ..
197     0
917     1
1349    1
1144    1
1458    1
Name: label, Length: 1195, dtype: int64

In [ ]:
y_test

205     0
694     0
183     0
1379    1
703     0
       ..
428     0
775     1
1130    1
1434    1
918     1
Name: label, Length: 299, dtype: int64

In [ ]:
#Tokenization
max_len = 100 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_text)

In [ ]:
#Counting the number of words
word_index = tokenizer.word_index
tot_words = len(word_index)
print('There are %s unique tokens in training data. ' % tot_words)

There are 4137 unique tokens in training data. 


In [ ]:
#Sequencing and padding
train_seq = tokenizer.texts_to_sequences(train_text)
train_pad = pad_sequences (train_seq, maxlen = max_len, padding = padding_type, truncating = trunc_type )


test_seq = tokenizer.texts_to_sequences(test_text)
test_pad = pad_sequences(test_seq, maxlen = max_len,
padding = padding_type, truncating = trunc_type)

In [ ]:
print('Shape of training tensor: ', train_pad.shape)
print('Shape of testing tensor: ', test_pad.shape)

Shape of training tensor:  (1195, 100)
Shape of testing tensor:  (299, 100)


In [ ]:
print(train_seq)

[[67, 74, 5, 1, 24, 139, 232, 56, 14, 4, 1, 25, 1, 1, 149, 270, 189, 62, 106, 255, 190, 1, 150, 90, 1], [35, 75, 1, 68, 25, 21, 170, 111, 65, 36, 22, 1, 2, 1, 171, 68, 16, 52, 31, 1, 12, 13, 127, 21, 420, 63, 68, 90, 39, 1, 1], [67, 74, 5, 1, 24, 139, 8, 1, 421, 25, 1, 149, 14, 1, 56, 270, 189, 255, 62, 106, 132, 1, 1, 1, 1], [140, 120, 1, 21, 1, 191, 1, 83, 1, 15, 1, 7, 1, 1, 103, 22, 103, 2, 233, 12, 63, 1, 296, 297, 234, 374, 462, 1], [1, 2, 69, 34, 1, 28, 77, 10, 16, 1, 3, 16, 1, 2, 1, 1, 1, 1, 48, 63, 1, 2, 1, 47, 58, 33, 2, 1], [256, 6, 375, 6, 1, 41, 1, 1, 271], [1, 57, 6, 422, 58, 15, 21, 1, 1, 1, 298], [74, 5, 45, 93, 79, 463, 18, 1, 1, 1, 1, 423, 1, 1, 76, 3, 20, 59, 4, 87, 141, 56, 14, 232, 49], [335, 206, 1, 5, 12, 1, 36, 1, 97, 107, 1, 317, 151, 132, 1, 1, 1, 1, 1, 222], [1, 1, 235, 8, 1, 1, 100, 4, 5, 24, 1, 257, 1, 5, 1, 2, 1, 8, 98], [192, 6, 70, 1, 46, 207, 464, 19, 29, 52, 35, 1], [318, 299, 79, 1, 1, 1, 1], [53, 3, 336, 4, 1, 1, 17, 133, 60, 61, 319, 128, 1, 26, 1, 4

# Model

In [ ]:
vocab_size = 500 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 24

In [ ]:
n_lstm = 20
drop_lstm =0.2

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
#model.add(GlobalAveragePooling1D())
#model.add(Dense(24, activation='relu'))
model.add(Bidirectional(tf.keras.layers.RNN(tf.keras.layers.LSTMCell(64))))
model.add(Dense(1, activation='sigmoid'))
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           8000      
                                                                 
 bidirectional (Bidirectiona  (None, 128)              41472     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 49,601
Trainable params: 49,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
num_epochs = 50
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history = model.fit(train_pad, y_train, epochs=num_epochs,
                    validation_data=(test_pad, y_test),
                    callbacks =[early_stop], verbose=2)

Epoch 1/50
38/38 - 42s - loss: 0.6290 - accuracy: 0.6594 - val_loss: 0.4467 - val_accuracy: 0.8261 - 42s/epoch - 1s/step
Epoch 2/50
38/38 - 24s - loss: 0.3711 - accuracy: 0.8519 - val_loss: 0.2452 - val_accuracy: 0.9064 - 24s/epoch - 619ms/step
Epoch 3/50
38/38 - 24s - loss: 0.1899 - accuracy: 0.9389 - val_loss: 0.1782 - val_accuracy: 0.9365 - 24s/epoch - 623ms/step
Epoch 4/50
38/38 - 24s - loss: 0.1213 - accuracy: 0.9598 - val_loss: 0.1264 - val_accuracy: 0.9632 - 24s/epoch - 643ms/step
Epoch 5/50
38/38 - 24s - loss: 0.1017 - accuracy: 0.9657 - val_loss: 0.1206 - val_accuracy: 0.9565 - 24s/epoch - 621ms/step
Epoch 6/50
38/38 - 25s - loss: 0.0775 - accuracy: 0.9774 - val_loss: 0.1491 - val_accuracy: 0.9431 - 25s/epoch - 653ms/step
Epoch 7/50
38/38 - 24s - loss: 0.0709 - accuracy: 0.9799 - val_loss: 0.1432 - val_accuracy: 0.9498 - 24s/epoch - 619ms/step
Epoch 8/50
38/38 - 24s - loss: 0.0547 - accuracy: 0.9866 - val_loss: 0.1406 - val_accuracy: 0.9532 - 24s/epoch - 627ms/step


The function called `predict_message` takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  msg= tokenizer.texts_to_sequences([pred_text])
  msg_pad=pad_sequences(msg, maxlen =max_len,
                      padding = padding_type,
                      truncating=trunc_type)
  pred=model.predict(msg_pad)
  l=len(pred)
  if pred[0]<0.5:
    prediction=[pred[0], 'ham']
  else:
    prediction=[pred[0], 'spam']



  return (prediction)



In [ ]:
# Run this cell to test your function and model.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    print(msg)
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


how are you doing today
1/1 [==============================] - 0s 400ms/step
[array([0.01471415], dtype=float32), 'ham']
sale today! to stop texts call 98912460324
1/1 [==============================] - 0s 53ms/step
[array([0.76375437], dtype=float32), 'spam']
i dont want to go. can we try it a different day? available sat
1/1 [==============================] - 0s 61ms/step
[array([0.01573204], dtype=float32), 'ham']
our new mobile video service is live. just install on your phone to start watching.
1/1 [==============================] - 0s 59ms/step
[array([0.9899512], dtype=float32), 'spam']
you have won £1000 cash! call to claim your prize.
1/1 [==============================] - 0s 60ms/step
[array([0.99571407], dtype=float32), 'spam']
i'll bring it tomorrow. don't forget the milk.
1/1 [==============================] - 0s 68ms/step
[array([0.00918701], dtype=float32), 'ham']
wow, is your arm alright. that happened to me one time too
1/1 [==============================] - 0s 60ms/st